# 乘法层的实现

In [12]:
#############################
class MulLayer:
    def __init__(self):
        #这里一定要写属性x和y是因为它们来保存正向传播时的输入值
        #加法层就不写，因为它不需要
        self.x = None
        self.y = None
        
    def forward(self , x , y):
        self.x = x
        self.y = y
        out = x*y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx,dy
###############################

In [13]:
# 使用乘法层实现苹果的购买
apple = 100
apple_num = 2
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_tax_layer= MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple , apple_num)
price = mul_tax_layer.forward(apple_price , tax)

print(price) #220

220.00000000000003


In [15]:
#用backward求导数
dprice = 1
dapple_price , dtax = mul_tax_layer.backward(dprice)
dapple , dapple_num = mul_apple_layer.backward(dapple_price)
#这两步真有点不太好理解，建议对着图看
#其实就是和正向传播的时候完全相反

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


![反向传播示例](img\img5_1.png " ")

# 加法层的实现

In [16]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self , x ,y):
        out = x + y
        
        return out
    
    def backward(self , dout):
        #dout保存上游传过来的值，上游是指右边
        dx = dout * 1
        dy = dout * 1
        
        return dx,dy
    

# 代码实现购买苹果和橘子的例子
![购买苹果和橘子](img\img5_2.png " ")

In [18]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
apple_plus_orange = AddLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple , apple_num)
orange_peice = mul_orange_layer.forward(orange , orange_num)
all_price = apple_plus_orange.forward(apple_price , orange_peice)
price = mul_tax_layer.forward(all_price , tax)

#backward
dprice = 1#第一个从上游传过来的参数是1

dall_price, dtax = mul_tax_layer.backward(dprice) #(4)
dapple_price, dorange_price = apple_plus_orange.backward(dall_price) #(3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) #(2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) #(1)

print(price)
print(dapple_num , dapple , dorange , dorange_num , dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


# 激活函数层的实现
把神经网络的层全都实现为类

## ReLU层
![ReLU函数](img\img5_3.png " ")
* 如果正向传播时的输入x大于0，则反向传播会将上游的值原封不动地传给下游
* 如果正向传播时的x小于等于0，则反向传播中传给下游的信号将停在此处

In [1]:
###########################
class Relu:
    def __init__(self):
        self.mask = None
        #mask是由True/False构成的NumPy数组
    
    def forward(self,x):
        #若x值小于0，则记录true；否则记录false
        self.mask = (x<=0)
        #out完全复制了x
        out = x.copy()
        #out这个数组上小于等于0的都赋值为0，大于等于0的就是本身
        out[self.mask] = 0
        
        return out
    
    def backward(self , dout):
        # 仅将小于等于0的值的梯度设置为零
        dout[self.mask] = 0
        #由于x>0时ReLU函数的导数为1，因此乘积简化为dout
        dx = dout

        return dx
###########################

## sigmoid层
![sigmoid函数](img\img5_4.png " ")
这里不仅有 * 节点和 + 节点，还有 exp 节点和 / 节点。

**sigmoid函数的分解：**

![步骤1](img\img5_5.png "")

![步骤2](img\img5_6.png "")

![步骤3](img\img5_7.png "")

![步骤3](img\img5_8.png "")